<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#isnull" data-toc-modified-id="isnull-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>isnull</a></span><ul class="toc-item"><li><span><a href="#na" data-toc-modified-id="na-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>na</a></span></li><li><span><a href="#outer" data-toc-modified-id="outer-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>outer</a></span></li></ul></li></ul></div>

In [1]:
import os, warnings
from distutils.sysconfig import get_python_lib,get_python_inc,get_python_version

In [2]:
# windows
# os.environ['SPARK_HOME']=f'{get_python_lib()}{os.sep}pyspark'
# os.environ['PYSPARK_PYTHON']=f'{get_python_lib()}{os.sep}pyspark'.split('Lib')[0]+'python.exe'
# linux
os.environ['SPARK_HOME']=f'{get_python_lib()}{os.sep}pyspark'
os.environ['PYSPARK_PYTHON']=get_python_lib().replace('lib','bin').replace('/site-packages','')

In [3]:
warnings.filterwarnings(action="ignore")

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types     import StructType, \
     StructField, FloatType, \
     IntegerType, StringType

In [5]:
referentiel = os.path.join(os.getcwd(),'warehouse')
os.makedirs(referentiel, exist_ok=True)

In [6]:
spark = SparkSession \
    .builder \
    .appName("PresentationSpark") \
    .config("spark.sql.warehouse.dir", referentiel) \
    .config("spark.executor.cores", 32) \
    .config("spark.executor.memory", '64g') \
    .enableHiveSupport() \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/22 17:38:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
spark

In [8]:
from pyspark.sql.functions import *
from pyspark.sql.types     import StructType, \
     StructField, FloatType, \
     IntegerType, StringType

meteoDataFrame  = spark.read.format('csv')\
    .option('sep',';')\
    .option('header','true')\
    .option('nullValue','mq')\
    .option('inferSchema', 'true')\
    .load('../donnees/meteo/*/') #.cache()

schema = StructType([
        StructField('Id'           , StringType() , True),
        StructField('ville'        , StringType() , True),
        StructField('latitude'     , FloatType() , True),
        StructField('longitude'    , FloatType() , True),
        StructField('altitude'     , IntegerType() , True)])

villes  = spark.read.format('csv')   \
      .option('sep',';')                \
      .option('mergeSchema', 'true')    \
      .option('header','true')          \
      .schema(schema)                   \
      .load('../donnees/postesSynop.csv')  #.cache()

@udf("string")
def formatVille(ville):
    if ville in ['CLERMONT-FD','MONT-DE-MARSAN',
                                   'ST-PIERRE','ST-BARTHELEMY METEO'] :
        return ville.title()
    else :
        if ville.find('-') != -1 :
            return ville[0:ville.find('-')].title()
        else:
            return ville.title()

villesT  = villes.select(
                col('Id').alias('id'),
                formatVille('ville').alias('ville'),
               'latitude',
               'longitude',
               'altitude')


meteo = meteoDataFrame.select(
                 col('numer_sta'),
                 col('date')[0:4].cast('int') ,
                 col('date')[5:2].cast('int'),
                 col('date')[7:2].cast('int'),
                 col('date')[5:4],
                 round(col('t') - 273.15,2),
                 col('u') / 100 ,
                 col('vv') / 1000 ,
                 col('pres') / 1000,
                 coalesce( col('rr3'),
                           col('rr24')/8,
                           col('rr12')/4,
                           col('rr6')/2,
                           col('rr1')*3  ) )\
             .toDF('id','annee','mois','jour','mois_jour','temperature',
                   'humidite','visibilite','pression','precipitations')\
             .cache()

meteo.select('annee','mois','jour','temperature','humidite',
             'visibilite','pression').toPandas().head(3)

,annee,mois,jour,temperature,humidite,visibilite,pression
0,2023,8,1,15.4,0.97,20.00,99.46
1,2023,8,1,15.6,0.95,51.23,99.63
2,2023,8,1,16.3,0.92,12.00,100.43


In [9]:
villesT.toPandas().head(3)

,id,ville,latitude,longitude,altitude
0,07005,Abbeville,50.136002,1.834000,69
1,07015,Lille,50.570000,3.097500,47
2,07020,Pte De La Hague,49.725166,-1.939833,6


# isnull

In [10]:
meteo.where('id < 8000').count()
description = meteo.where('id < 8000')\
     .select('annee','temperature', 'humidite',
             'visibilite', 'pression','precipitations')\
     .describe()\
     .select([c if c == 'summary'
                else round(c,2).alias(c)
                for c in
                    ['summary','annee','temperature', 'humidite',
                   'visibilite', 'pression','precipitations']])\
     .toPandas()

In [11]:
meteo.where('id < 8000').where(meteo['temperature'].isNull()).count()

14342

In [12]:
meteo.where('id < 8000').where(meteo['humidite'].isNull()).count()

18041

In [13]:
meteo.where('id < 8000').where(meteo['visibilite'].isNull()).count()

442045

In [14]:
meteo.where('id < 8000').where(meteo['pression'].isNull()).count()

20811

In [15]:
meteo.where('id < 8000').where(meteo['precipitations'].isNull()).count()

29308

In [16]:
meteo.where('id < 8000').toPandas().isna().sum()

id                     0
annee                  0
mois                   0
jour                   0
mois_jour              0
temperature        14342
humidite           18041
visibilite        442045
pression           20811
precipitations     29308
dtype: int64

In [ ]:
meteoDataFrame.toPandas().isna().sum()

In [18]:
# from databricks import koalas as ks
meteoDataFrame.to_koalas().isna().sum()

ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
meteo.where('id < 8000')\
     .where(meteo['temperature'].isNotNull())\
     .where(meteo['humidite'].isNotNull() )\
     .where(meteo['visibilite'].isNotNull() )\
     .where(meteo['pression'].isNotNull() )\
     .count()

## na

In [ ]:
meteo.where('id < 8000')\
     .na.fill(0 ,["precipitations"])\
     .na.drop()\
     .count()

In [ ]:
meteo = meteoDataFrame.select(
                 col('numer_sta'),
                 col('date')[0:4].cast('int') ,
                 col('date')[5:2].cast('int'),
                 col('date')[7:2].cast('int'),
                 col('date')[5:4],
                 round(col('t') - 273.15,2),
                 col('u') / 100 ,
                 col('vv') / 1000 ,
                 col('pres') / 1000,
                 col('rr1')*3,
                 col('rr3'),
                 col('rr6')/2,
                 col('rr12')/4,
                 col('rr24')/8)\
                   .toDF('id','annee','mois','jour','mois_jour','temperature',
                   'humidite','visibilite','pression',
                   'precipitations1','precipitations3','precipitations6',
                   'precipitations12','precipitations24')\
             .cache()

In [ ]:
meteo.where('id < 8000')\
       .select('precipitations1','precipitations3','precipitations6',
             'precipitations12','precipitations24','temperature')\
       .toDF('prec1','prec3','prec6','prec12','prec24','temp')\
       .describe()\
       .select([c if c == 'summary'
                else round(c,2).alias(c)
                for c in['prec1','prec3','prec6',
                          'prec12','prec24','temp']])\
      .toPandas()

In [ ]:
meteo.select( coalesce('precipitations3','precipitations24',
                       'precipitations12','precipitations6',
                       'precipitations1').alias('precipitations')).toPandas().head(20)

In [ ]:
meteo.where('id < 8000')\
     .select( coalesce('precipitations24','precipitations12',
                       'precipitations6','precipitations3',
                       'precipitations1').alias('precipitationsH')
            ).describe().toPandas()

In [ ]:
meteo.to_koalas().isna().sum()

In [ ]:
meteo.where('id < 8000').count()